In [36]:
import pandas as pd
import pulp

In [37]:
demand_df = pd.read_csv("data/demand.csv")
vehicles_df = pd.read_csv("top_ranked_combined.csv")
carbon_limits_df = pd.read_csv("data/carbon_emissions.csv")
current_fleet = pd.read_csv("current_fleet.csv")

In [38]:
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         256 non-null    int64 
 1   Size         256 non-null    object
 2   Distance     256 non-null    object
 3   Demand (km)  256 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 8.1+ KB


In [39]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Operating Year           256 non-null    int64  
 1   Size                     256 non-null    object 
 2   Distance_demand          256 non-null    object 
 3   Demand (km)              256 non-null    int64  
 4   ID                       256 non-null    object 
 5   Vehicle                  256 non-null    object 
 6   Available Year           256 non-null    int64  
 7   Cost ($)                 256 non-null    int64  
 8   Yearly range (km)        256 non-null    int64  
 9   Distance_vehicle         256 non-null    object 
 10  Fuel                     256 non-null    object 
 11  carbon_emissions_per_km  256 non-null    float64
 12  insurance_cost           256 non-null    float64
 13  maintenance_cost         256 non-null    float64
 14  fuel_costs_per_km        2

In [40]:
current_fleet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Size_Bucket      16 non-null     object 
 1   Distance_Bucket  16 non-null     object 
 2   IDs              0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 516.0+ bytes


In [41]:
carbon_limits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Year                    16 non-null     int64
 1   Carbon emission CO2/kg  16 non-null     int64
dtypes: int64(2)
memory usage: 388.0 bytes


In [44]:
# Initialize the problem
prob = pulp.LpProblem("Fleet_Optimization", pulp.LpMinimize)

# Decision variables
fleet = pulp.LpVariable.dicts("Fleet", [(year, size, dist, vehicle_id) 
                                         for year in range(2023, 2039)
                                         for size in demand_df['Size'].unique()
                                         for dist in demand_df['Distance'].unique()
                                         for vehicle_id in vehicles_df['ID'].unique()],
                             lowBound=0, cat='Integer')

buy = pulp.LpVariable.dicts("Buy", [(year, size, dist, vehicle_id)
                                     for year in range(2023, 2039)
                                     for size in demand_df['Size'].unique()
                                     for dist in demand_df['Distance'].unique()
                                     for vehicle_id in vehicles_df['ID'].unique()],
                            lowBound=0, cat='Integer')

sell = pulp.LpVariable.dicts("Sell", [(year, size, dist, vehicle_id)
                                       for year in range(2023, 2039)
                                       for size in demand_df['Size'].unique()
                                       for dist in demand_df['Distance'].unique()
                                       for vehicle_id in vehicles_df['ID'].unique()],
                              lowBound=0, cat='Integer')

# Objective function: Minimize total costs
prob += pulp.lpSum([(vehicles_df.loc[vehicles_df['ID'] == vehicle_id, 'Cost ($)'].values[0] + 
                     vehicles_df.loc[vehicles_df['ID'] == vehicle_id, 'insurance_cost'].values[0] + 
                     vehicles_df.loc[vehicles_df['ID'] == vehicle_id, 'maintenance_cost'].values[0]) * buy[(year, size, dist, vehicle_id)]
                    for year, size, dist, vehicle_id in buy.keys()])

# Constraints
for year in range(2023, 2039):
    for size, dist in zip(demand_df['Size'], demand_df['Distance']):
        yearly_demand = demand_df[(demand_df['Year'] == year) & (demand_df['Size'] == size) & (demand_df['Distance'] == dist)]['Demand (km)'].values[0]
        carbon_limit = carbon_limits_df[carbon_limits_df['Year'] == year]['Carbon emission CO2/kg'].values[0]
        
        # Demand constraint
        prob += pulp.lpSum([fleet[(year, size, dist, vehicle_id)] * 
                            vehicles_df.loc[vehicles_df['ID'] == vehicle_id, 'Yearly range (km)'].values[0]
                            for vehicle_id in vehicles_df['ID'].unique()]) >= yearly_demand
        
        # Carbon emission constraint
        prob += pulp.lpSum([fleet[(year, size, dist, vehicle_id)] * 
                            vehicles_df.loc[vehicles_df['ID'] == vehicle_id, 'carbon_emissions_per_km'].values[0] * 
                            yearly_demand 
                            for vehicle_id in vehicles_df['ID'].unique()]) <= carbon_limit
        
        # Fleet balance constraint
        if year == 2023:
            # No previous fleet, all vehicles must be bought
            prob += pulp.lpSum([fleet[(year, size, dist, vehicle_id)] for vehicle_id in vehicles_df['ID'].unique()]) == \
                    pulp.lpSum([buy[(year, size, dist, vehicle_id)] for vehicle_id in vehicles_df['ID'].unique()])
        else:
            # Ensure previous year fleet exists, otherwise default to zero
            prob += pulp.lpSum([fleet.get((year, size, dist, vehicle_id), 0) for vehicle_id in vehicles_df['ID'].unique()]) == \
                    pulp.lpSum([fleet.get((year-1, size, dist, vehicle_id), 0) - sell[(year, size, dist, vehicle_id)] + buy[(year, size, dist, vehicle_id)]
                                for vehicle_id in vehicles_df['ID'].unique()])
        
        # Selling limit (only apply if there’s a previous fleet)
        if year > 2023:
            prob += pulp.lpSum([sell[(year, size, dist, vehicle_id)] for vehicle_id in vehicles_df['ID'].unique()]) <= \
                    0.2 * pulp.lpSum([fleet.get((year-1, size, dist, vehicle_id), 0) for vehicle_id in vehicles_df['ID'].unique()])
        
        # Vehicle lifetime constraint (10 years)
        for vehicle_id in vehicles_df['ID'].unique():
            available_year = vehicles_df.loc[vehicles_df['ID'] == vehicle_id, 'Available Year'].values[0]
            if year - available_year >= 10:
                prob += fleet[(year, size, dist, vehicle_id)] == 0

# Solve the problem
prob.solve()

# Export results
fleet_plan = []
for v in prob.variables():
    if v.varValue > 0 and 'Fleet' in v.name:
        parts = v.name.split("_")
        if len(parts) == 5:
            _, year, size, dist, vehicle_id = parts
            fleet_plan.append([int(year), size, dist, vehicle_id, int(v.varValue)])
        else:
            print(f"Unexpected variable format: {v.name}")

fleet_df = pd.DataFrame(fleet_plan, columns=['Year', 'Size', 'Distance', 'Vehicle ID', 'Count'])
fleet_df.to_csv('fleet_plan.csv', index=False)

print("Optimization complete. Fleet plan exported.")

# Let me know if you’d like me to tweak anything or explain the code further! 🚀

Unexpected variable format: Fleet_(2023,_'S1',_'D1',_'BEV_S4_2035')
Unexpected variable format: Fleet_(2023,_'S1',_'D2',_'BEV_S4_2038')
Unexpected variable format: Fleet_(2023,_'S1',_'D3',_'BEV_S4_2034')
Unexpected variable format: Fleet_(2023,_'S1',_'D4',_'BEV_S4_2035')
Unexpected variable format: Fleet_(2023,_'S2',_'D1',_'BEV_S4_2035')
Unexpected variable format: Fleet_(2023,_'S2',_'D2',_'BEV_S4_2038')
Unexpected variable format: Fleet_(2023,_'S2',_'D3',_'BEV_S4_2038')
Unexpected variable format: Fleet_(2023,_'S2',_'D4',_'BEV_S4_2033')
Unexpected variable format: Fleet_(2023,_'S3',_'D1',_'BEV_S4_2036')
Unexpected variable format: Fleet_(2023,_'S3',_'D2',_'BEV_S4_2036')
Unexpected variable format: Fleet_(2023,_'S3',_'D3',_'BEV_S4_2038')
Unexpected variable format: Fleet_(2023,_'S3',_'D4',_'BEV_S4_2033')
Unexpected variable format: Fleet_(2023,_'S4',_'D1',_'LNG_S4_2032')
Unexpected variable format: Fleet_(2023,_'S4',_'D2',_'BEV_S4_2038')
Unexpected variable format: Fleet_(2023,_'S4',_'

In [8]:
model = pulp.LpProblem("Fleet_Optimization", pulp.LpMinimize)

In [9]:
# Decision variables
buy_vars = {}   # Vehicles bought each year
use_vars = {}   # Vehicles in use each year
sell_vars = {}  # Vehicles sold each year

In [24]:
vehicles_topsis_combinations = vehicles_df.groupby(['Operating Year', 'Size', 'Distance_demand'])

In [25]:
current_fleet_combinations = current_fleet.groupby(['Size_Bucket', 'Distance_Bucket'])

In [45]:
for (year, size, distance), group in vehicles_topsis_combinations:
    vehicle_id_list = group["ID"].tolist()  # Extract vehicle IDs from the group

    # Initialize the year if not already in dictionary
    if year.item() not in buy_vars:
        buy_vars[year.item()] = {}

    # Initialize the (size, distance) key within the year
    if f"{size}_{distance}" not in buy_vars[year.item()]:
        buy_vars[year.item()][f"{size}_{distance}"] = {}

    # Create decision variables for each vehicle ID in this (size, distance) combination
    for v_id in vehicle_id_list:
        buy_vars[year][f"{size}_{distance}"] = pulp.LpVariable(f"{v_id}", lowBound=0, cat="Integer")


In [46]:
for index, row in current_fleet.iterrows():
    size = row["Size_Bucket"]
    distance = row["Distance_Bucket"]
    v_id = row["IDs"]
    sell_vars[f"{size}_{distance}"] = pulp.LpVariable(f"{v_id}", lowBound=0, cat="Integer")

In [47]:
buy_vars

{2023: {'S1_D1': LNG_S1_2023,
  'S1_D2': Diesel_S1_2023,
  'S1_D3': Diesel_S1_2023,
  'S1_D4': Diesel_S1_2023,
  'S2_D1': LNG_S2_2023,
  'S2_D2': Diesel_S2_2023,
  'S2_D3': Diesel_S2_2023,
  'S2_D4': Diesel_S2_2023,
  'S3_D1': LNG_S3_2023,
  'S3_D2': Diesel_S3_2023,
  'S3_D3': Diesel_S3_2023,
  'S3_D4': Diesel_S3_2023,
  'S4_D1': LNG_S4_2023,
  'S4_D2': Diesel_S4_2023,
  'S4_D3': Diesel_S4_2023,
  'S4_D4': Diesel_S4_2023},
 2024: {'S1_D1': LNG_S1_2024,
  'S1_D2': Diesel_S1_2024,
  'S1_D3': Diesel_S1_2024,
  'S1_D4': Diesel_S1_2024,
  'S2_D1': LNG_S2_2024,
  'S2_D2': Diesel_S2_2024,
  'S2_D3': Diesel_S2_2024,
  'S2_D4': Diesel_S2_2024,
  'S3_D1': LNG_S3_2024,
  'S3_D2': Diesel_S3_2024,
  'S3_D3': Diesel_S3_2024,
  'S3_D4': Diesel_S3_2024,
  'S4_D1': LNG_S4_2024,
  'S4_D2': Diesel_S4_2024,
  'S4_D3': Diesel_S4_2024,
  'S4_D4': Diesel_S4_2024},
 2025: {'S1_D1': LNG_S1_2025,
  'S1_D2': Diesel_S1_2025,
  'S1_D3': Diesel_S1_2025,
  'S1_D4': Diesel_S1_2025,
  'S2_D1': LNG_S2_2025,
  'S2_D2': 

In [37]:
def create_objective_function(model, buy_vars, use_vars, sell_vars, vehicles_df, years_range):
    """
    Creates the objective function for fleet optimization problem
    
    Parameters:
    - model: pulp.LpProblem object
    - buy_vars: Dictionary of purchase decision variables
    - use_vars: Dictionary of usage decision variables
    - sell_vars: Dictionary of sell decision variables
    - vehicles_df: DataFrame containing vehicle information
    - years_range: Range of years to consider
    """
    
    # Initialize total cost
    total_cost = 0
    
    # Purchase costs
    for year in years_range:
        for size_distance, vehicles in buy_vars[year].items():
            for v_id, var in vehicles.items():
                # Get purchase cost for this vehicle from vehicles_df
                purchase_cost = vehicles_df.loc[vehicles_df['ID'] == v_id, 'Cost ($)'].values[0]
                total_cost += purchase_cost * var

    # # Operating costs for vehicles in use
    # for year in years_range:
    #     for size_distance, vehicles in use_vars[year].items():
    #         for v_id, var in vehicles.items():
    #             # Get operating cost for this vehicle from vehicles_df
    #             operating_cost = vehicles_df.loc[vehicles_df['ID'] == v_id, 'Operating_Cost'].values[0]
    #             yearly_distance = vehicles_df.loc[vehicles_df['ID'] == v_id, 'Yearly_Distance'].values[0]
    #             total_cost += operating_cost * yearly_distance * var

    # # Salvage value (negative cost) for sold vehicles
    # for size_distance, vehicles in sell_vars.items():
    #     for v_id, var in vehicles.items():
    #         # Get salvage value for this vehicle from vehicles_df
    #         # Assuming salvage value is a percentage of purchase cost
    #         purchase_cost = vehicles_df.loc[vehicles_df['ID'] == v_id, 'Purchase_Cost'].values[0]
    #         salvage_value = 0.2 * purchase_cost  # Example: 20% of purchase cost
    #         total_cost -= salvage_value * var

    # Set the objective function
    model += total_cost

    return model

In [38]:
model = create_objective_function(model, buy_vars, use_vars, sell_vars, vehicles_df, range(2025, 2039))